## 

In [4]:
# Import dependencies

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import sqlite3
from flask import Flask, jsonify
import pandas as pd

# Set path Geckodriver executable
geckodriver_path = "\\Users\\edwar\\anaconda3\\Lib\\site-packages\\selenium\\webdriver\\firefox\\geckodriver.exe"  

# Configure Firefox options
firefox_options = Options()
firefox_options.headless = True  

# Set Firefox web driver
firefox_service = FirefoxService(geckodriver_path)
driver = webdriver.Firefox(service=firefox_service, options=firefox_options)

# Url scrape
url = "https://steamdb.info/charts/"

# Load the page
driver.get(url)

# Add a delay of 3 seconds to allow the table to initialize
time.sleep(3)

wait = WebDriverWait(driver, 30)  # Increased the wait time to 30 seconds

# Wait for the table to become visible using XPath and wait for "Initializing table" message to disappear
try:
    wait.until_not(EC.visibility_of_element_located((By.XPATH, "//div[text()='Initializing table']")))
except TimeoutException:
    print("Table initialization timed out.")
    driver.quit()

data_list = []

# Scrape data from the first sheet (button "1" is already selected by default)
table = driver.find_element(By.CSS_SELECTOR, "table.table-products")
rows = table.find_elements(By.XPATH, "//table[@id='table-apps']/tbody/tr")
for row in rows:
    columns = row.find_elements(By.TAG_NAME, "td")
    number = columns[0].text.strip()
    name = columns[2].text.strip()
    current = columns[3].text.strip()
    peak_24h = columns[4].text.strip()
    peak_all_time = columns[5].text.strip()

    game_data = {
        "Number": number,
        "Name": name,
        "Current": current,
        "24h Peak": peak_24h,
        "All-Time Peak": peak_all_time,
    }

    data_list.append(game_data)

# Click the "2" link to switch to the second sheet
try:
    link_2 = wait.until(EC.element_to_be_clickable((By.LINK_TEXT, '2')))
    link_2.click()
except TimeoutException:
    print("Link '2' not found or clickable.")
    driver.quit()

# Add a delay to allow the second sheet to load
time.sleep(3)  # Adjust the delay as needed

# Scrape data from the second sheet
table = driver.find_element(By.CSS_SELECTOR, "table.table-products")
rows = table.find_elements(By.XPATH, "//table[@id='table-apps']/tbody/tr")
for row in rows:
    columns = row.find_elements(By.TAG_NAME, "td")
    number = columns[0].text.strip()
    name = columns[2].text.strip()
    current = columns[3].text.strip()
    peak_24h = columns[4].text.strip()
    peak_all_time = columns[5].text.strip()

    game_data = {
        "Number": number,
        "Name": name,
        "Current": current,
        "24h Peak": peak_24h,
        "All-Time Peak": peak_all_time,
    }

    data_list.append(game_data)

# Close the browser
driver.quit()

# Now, data_list contains the scraped data from both sheets
for game in data_list:
    print(game)


C:\Users\edwar\AppData\Local\Temp\ipykernel_27656\1376288943.py:19: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  firefox_options.headless = True
C:\Users\edwar\AppData\Local\Temp\ipykernel_27656\1376288943.py:22: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  firefox_service = FirefoxService(geckodriver_path)


{'Number': '1.', 'Name': 'Counter-Strike: Global Offensive', 'Current': '804,976', '24h Peak': '1,285,148', 'All-Time Peak': '1,818,773'}
{'Number': '2.', 'Name': 'Dota 2', 'Current': '265,236', '24h Peak': '651,740', 'All-Time Peak': '1,295,114'}
{'Number': '3.', 'Name': "Baldur's Gate 3", 'Current': '178,108', '24h Peak': '284,503', 'All-Time Peak': '875,343'}
{'Number': '4.', 'Name': 'Cyberpunk 2077', 'Current': '158,208', '24h Peak': '167,465', 'All-Time Peak': '1,054,388'}
{'Number': '5.', 'Name': 'Apex Legends', 'Current': '146,867', '24h Peak': '397,605', 'All-Time Peak': '624,473'}
{'Number': '6.', 'Name': 'PUBG: BATTLEGROUNDS', 'Current': '127,923', '24h Peak': '382,200', 'All-Time Peak': '3,257,248'}
{'Number': '7.', 'Name': 'NARAKA: BLADEPOINT', 'Current': '86,125', '24h Peak': '217,063', 'All-Time Peak': '264,406'}
{'Number': '8.', 'Name': 'Starfield', 'Current': '76,233', '24h Peak': '129,187', 'All-Time Peak': '330,723'}
{'Number': '9.', 'Name': 'Source SDK Base 2007', 'C

In [5]:
# SQLite Database

# Create a connection to the SQLite database 
conn = sqlite3.connect("steam_data.db")

# Create a cursor object
cursor = conn.cursor()

In [6]:
# Define the table schema
cursor.execute('''
    CREATE TABLE IF NOT EXISTS games (
        Number TEXT,
        Name TEXT,
        Current TEXT,
        Peak_24h TEXT,
        Peak_All_Time TEXT
    )
''')

In [7]:
# Insert data into the table
for game in data_list:
    cursor.execute('''
        INSERT INTO games (Number, Name, Current, Peak_24h, Peak_All_Time)
        VALUES (?, ?, ?, ?, ?)
    ''', (game["Number"], game["Name"], game["Current"], game["24h Peak"], game["All-Time Peak"]))

In [8]:
# Commit changes
conn.commit()
cursor.close()
conn.close()


In [9]:
# Create a DataFrame from the SQLite database
conn = sqlite3.connect("steam_data.db")
webscraped_games = pd.read_sql_query("SELECT * FROM games", conn)
conn.close()
webscraped_games

,Number,Name,Current,Peak_24h,Peak_All_Time
0,1.,Counter-Strike: Global Offensive,"963,043","1,410,713","1,818,773"
1,2.,Dota 2,"388,645","716,286","1,295,114"
2,3.,Apex Legends,"264,291","454,151","624,473"
3,4.,Baldur's Gate 3,"246,991","360,993","875,343"
4,5.,PUBG: BATTLEGROUNDS,"208,802","400,528","3,257,248"
...,...,...,...,...,...
2795,196.,SCP: Secret Laboratory,"2,587","6,964","11,484"
2796,197.,Grounded,"2,542","4,933","32,012"
2797,198.,Stumble Guys,"2,476","13,268","43,705"
2798,199.,MyDockFinder,"2,452","3,539","3,869"


In [10]:

# Connect to the SQLite database
conn = sqlite3.connect("steam_data.db")
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='games';")

# Fetch the result
result = cursor.fetchone()

# Check if the result is not None (table exists)
if result:
    print("The 'games' table exists in the database.")
else:
    print("The 'games' table does not exist in the database.")

# Close the cursor (optional, but recommended)
cursor.close()

# Close the database connection
conn.close()



The 'games' table exists in the database.


In [58]:
#Import dependencies (here b/c putting it in box1 did not work for my computer)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
from pathlib import Path

# Files to Load
steam_games_to_load = Path("raw data/steam_games_updated.csv")

#Read Data and store into Pandas DataFrames
steam_games = pd.read_csv(steam_games_to_load)
steam_games.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/952950/0304/,app,3.04,04/03/2017How much does this date tell you? Fo...,NaN,"Mostly Positive,(80),- 76% of the 80 user revi...","Jan 1, 2019",JPEG OF PAIN GAMES,"JPEG OF PAIN,JPEG OF PAIN","Adventure,Action,Indie,Horror,FPS,Survival Horror","Single-player,Steam Achievements,Profile Featu...","English,Russian",16,"Action,Adventure,Indie",About This Game We all die. Heaven is waiting...,NaN,"Minimum:,OS:,Windows 7, 8, 8.1, 10 (x32 | x64)...","Recommended:,OS:,Windows 7, 8, 8.1, 10 (x64),P...",PROLOGUE,NaN
1,https://store.steampowered.com/app/684210/7/,app,7,"7 is a puzzle platformer, literally, by the nu...",NaN,"Very Positive,(114),- 81% of the 114 user revi...","Jan 25, 2018",Zac,"Zac,Zac","Free to Play,Indie,Adventure","Single-player,Partial Controller Support",English,10,"Adventure,Free to Play,Indie","About This Game 7 is a single-player, 2-D, pu...",NaN,NaN,NaN,Free,NaN
2,https://store.steampowered.com/app/578850/640/,app,64,"64.0 is a minimal fast-paced arcade game, wher...",NaN,"Very Positive,(803),- 90% of the 803 user revi...","Mar 16, 2017",rebel rabbit,"indienova,GrabTheGames,indienova,GrabTheGames","Indie,Difficult,Action,Rhythm,Retro,Great Soun...","Single-player,Steam Achievements,Steam Trading...","English,French,Italian,German,Spanish - Spain,...",14,"Action,Indie",About This Game Beware! This game is really ...,NaN,"Minimum:,OS:,Microsoft® Windows® XP / Vista / ...","Recommended:,OS:,Microsoft® Windows® XP / Vist...",$2.99,$11.36
3,https://store.steampowered.com/app/1016250/100/,app,100$,Money rules the world! This is a casual puzzle...,NaN,"Positive,(28),- 89% of the 28 user reviews for...","Feb 1, 2019",КиКо,"КиКо,КиКо",Indie,"Single-player,Steam Achievements,Full controll...","English,French,Italian,German,Spanish - Spain,...",100,Indie,About This Game The fate of a lost wallet is ...,NaN,NaN,NaN,$1.98,$1.18
4,https://store.steampowered.com/app/1036910/100/,app,€100,The Magic Wallet once again take his quest in ...,NaN,"Positive,(16),- 81% of the 16 user reviews for...","Mar 18, 2019",КиКо,"КиКо,КиКо",Casual,"Single-player,Steam Achievements,Full controll...","English,French,Italian,German,Spanish - Spain,...",100,Casual,About This Game After wandering through many...,NaN,NaN,NaN,$1.98,$1.18


In [59]:
#Pull out the relevant columns
steam_games_df = steam_games[['name','types','developer','publisher','popular_tags','genre','original_price']].copy()

In [60]:
#Remove bundles
steam_games_df = steam_games_df.loc[steam_games_df['types']!='bundle']

#Remove rows with empty values/ NaN strings
steam_games_df.dropna(inplace = True)
steam_games_df

,name,types,developer,publisher,popular_tags,genre,original_price
0,3.04,app,JPEG OF PAIN GAMES,"JPEG OF PAIN,JPEG OF PAIN","Adventure,Action,Indie,Horror,FPS,Survival Horror","Action,Adventure,Indie",PROLOGUE
1,7,app,Zac,"Zac,Zac","Free to Play,Indie,Adventure","Adventure,Free to Play,Indie",Free
2,64,app,rebel rabbit,"indienova,GrabTheGames,indienova,GrabTheGames","Indie,Difficult,Action,Rhythm,Retro,Great Soun...","Action,Indie",$2.99
3,100$,app,КиКо,"КиКо,КиКо",Indie,Indie,$1.98
4,€100,app,КиКо,"КиКо,КиКо",Casual,Casual,$1.98
...,...,...,...,...,...,...,...
40948,Forza Horizon 4,app,Playground Games,Xbox Game Studios,"Racing, Open World, Driving, Multiplayer, Auto...",Racing,$59.99
40949,STAR WARS™: The Old Republic™,app,Broadsword,Electronic Arts,"Free to Play, Multiplayer, MMORPG, Character C...","Massively Multiplayer, RPG, Free to Play",Free
40950,Grounded,app,Obsidian Entertainment,Xbox Game Studios,"Survival, Multiplayer, Open World Survival Cra...","Action, Adventure",$39.99
40951,Stumble Guys,app,Scopely,Scopely,"Free to Play, Multiplayer, Casual, Action, 3D,...","Action, Casual, Indie",Free


In [62]:
#Split up the genre tags into a new DF
genre_df = steam_games_df['genre'].str.split(',', expand=True)

#Rename Columns so they make sense
genre_df.columns = ['Genre1', 'Genre2', 'Genre3', 'Genre4', 'Genre5', 'Genre6', 'Genre7', 'Genre8', 'Genre9', 'Genre10', 'Genre11', 'Genre12', 'Genre13']

#Add in name of the game
genre_df.insert(0,'name',steam_games_df['name'])

#Sort the genres alphabetically
genre_df.sort_values(by= 
                     ['Genre1', 'Genre2', 'Genre3', 'Genre4', 'Genre5', 'Genre6', 'Genre7', 'Genre8', 'Genre9', 'Genre10', 'Genre11', 'Genre12', 'Genre13'], axis=0, ascending=True, inplace=True, kind='quicksort', na_position='last', ignore_index=True, key=None)

genre_df

,name,Genre1,Genre2,Genre3,Genre4,Genre5,Genre6,Genre7,Genre8,Genre9,Genre10,Genre11,Genre12,Genre13
0,"MovieMator Video Editor Pro - Movie Maker, Vid...",Accounting,Animation & Modeling,Audio Production,Design & Illustration,Education,Photo Editing,Software Training,Utilities,Video Production,Web Publishing,Game Development,None,None
1,VeeR VR:VR Video and Movie Platform,Accounting,Animation & Modeling,Audio Production,Design & Illustration,Education,Photo Editing,Software Training,Utilities,Video Production,Web Publishing,Game Development,None,None
2,Multiplicity,Accounting,Animation & Modeling,Audio Production,Design & Illustration,Education,Photo Editing,Software Training,Utilities,Video Production,Web Publishing,None,None,None
3,DatavizVR Demo,Accounting,Education,Software Training,Utilities,Early Access,None,None,None,None,None,None,None,None
4,Aimlabs,Action,Adventure,Casual,Free to Play,Indie,Simulation,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30173,Visual Novel Maker - Symphonic Adventure Music...,Web Publishing,None,None,None,None,None,None,None,None,None,None,None,None
30174,Visual Novel Maker - TeikokukaigunKoibojo Coll...,Web Publishing,None,None,None,None,None,None,None,None,None,None,None,None
30175,Visual Novel Maker - TOKIWA GRAPHICS Event BG ...,Web Publishing,None,None,None,None,None,None,None,None,None,None,None,None
30176,Visual Novel Maker - TOKIWA GRAPHICS Event BG ...,Web Publishing,None,None,None,None,None,None,None,None,None,None,None,None


In [63]:
#Split up the Popular Tags into a new DF
popular_tags_df = steam_games_df['popular_tags'].str.split(',', expand=True)

#Rename Columns so they make sense
popular_tags_df.columns = ['PopTag1', 'PopTag2', 'PopTag3', 'PopTag4', 'PopTag5', 'PopTag6', 'PopTag7', 'PopTag8', 'PopTag9', 'PopTag10', 
                           'PopTag11', 'PopTag12', 'PopTag13', 'PopTag14', 'PopTag15', 'PopTag16', 'PopTag17', 'PopTag18', 'PopTag19', 'PopTag20']

#Add in name of the game
popular_tags_df.insert(0,'name',steam_games_df['name'])

popular_tags_df

,name,PopTag1,PopTag2,PopTag3,PopTag4,PopTag5,PopTag6,PopTag7,PopTag8,PopTag9,...,PopTag11,PopTag12,PopTag13,PopTag14,PopTag15,PopTag16,PopTag17,PopTag18,PopTag19,PopTag20
0,3.04,Adventure,Action,Indie,Horror,FPS,Survival Horror,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,7,Free to Play,Indie,Adventure,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,64,Indie,Difficult,Action,Rhythm,Retro,Great Soundtrack,Pixel Graphics,Abstract,2D,...,Singleplayer,Top-Down Shooter,Arcade,Replay Value,Shooter,Survival,Top-Down,Minimalist,Music,None
3,100$,Indie,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,€100,Casual,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40948,Forza Horizon 4,Racing,Open World,Driving,Multiplayer,Automobile Sim,Online Co-Op,Realistic,Singleplayer,Controller,...,None,None,None,None,None,None,None,None,None,None
40949,STAR WARS™: The Old Republic™,Free to Play,Multiplayer,MMORPG,Character Customization,Open World,RPG,Story Rich,Massively Multiplayer,None,...,None,None,None,None,None,None,None,None,None,None
40950,Grounded,Survival,Multiplayer,Open World Survival Craft,Base Building,Co-op,Crafting,Adventure,Open World,Horror,...,None,None,None,None,None,None,None,None,None,None
40951,Stumble Guys,Free to Play,Multiplayer,Casual,Action,3D,3D Platformer,Battle Royale,Colorful,Family Friendly,...,None,None,None,None,None,None,None,None,None,None


In [64]:
#Clean up the prices column

#Change anything with Free to $0.00
steam_games_df['original_price'] = steam_games_df['original_price'].apply(lambda x: '$0.00' if 'Free' in x else x)

#Change anything with Demo to $0.00
steam_games_df['original_price'] = steam_games_df['original_price'].apply(lambda x: '$0.00' if 'Demo' in x else x)

#Drop any rows that do not contain a $ (errant cells that have random text)
steam_games_df = steam_games_df[steam_games_df['original_price'].str.contains('\$', na=False)]

#Drop the $ and convert to numbers
steam_games_df['original_price'] = steam_games_df['original_price'].str.replace('$', '').astype(float)

steam_games_df

C:\Users\edwar\AppData\Local\Temp\ipykernel_27656\1465947271.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  steam_games_df['original_price'] = steam_games_df['original_price'].str.replace('$', '').astype(float)
C:\Users\edwar\AppData\Local\Temp\ipykernel_27656\1465947271.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_games_df['original_price'] = steam_games_df['original_price'].str.replace('$', '').astype(float)


,name,types,developer,publisher,popular_tags,genre,original_price
1,7,app,Zac,"Zac,Zac","Free to Play,Indie,Adventure","Adventure,Free to Play,Indie",0.00
2,64,app,rebel rabbit,"indienova,GrabTheGames,indienova,GrabTheGames","Indie,Difficult,Action,Rhythm,Retro,Great Soun...","Action,Indie",2.99
3,100$,app,КиКо,"КиКо,КиКо",Indie,Indie,1.98
4,€100,app,КиКо,"КиКо,КиКо",Casual,Casual,1.98
5,103,app,Dystopia Interactive,"Dystopia Interactive,Dystopia Interactive","Adventure,Casual,Indie,Horror","Adventure,Casual,Indie",9.99
...,...,...,...,...,...,...,...
40948,Forza Horizon 4,app,Playground Games,Xbox Game Studios,"Racing, Open World, Driving, Multiplayer, Auto...",Racing,59.99
40949,STAR WARS™: The Old Republic™,app,Broadsword,Electronic Arts,"Free to Play, Multiplayer, MMORPG, Character C...","Massively Multiplayer, RPG, Free to Play",0.00
40950,Grounded,app,Obsidian Entertainment,Xbox Game Studios,"Survival, Multiplayer, Open World Survival Cra...","Action, Adventure",39.99
40951,Stumble Guys,app,Scopely,Scopely,"Free to Play, Multiplayer, Casual, Action, 3D,...","Action, Casual, Indie",0.00


In [65]:
#Merge the main df with the genre df and then merge that with the popular tags df
cleaned_steam_games = pd.merge(steam_games_df, genre_df, on='name', how='inner')
cleaned_steam_games = pd.merge(cleaned_steam_games, popular_tags_df, on='name', how='inner')

#Drop the genre and popular tag columns now that theyve been expanded                              
cleaned_steam_games = cleaned_steam_games.drop('genre', axis=1)
cleaned_steam_games = cleaned_steam_games.drop('popular_tags', axis=1)
cleaned_steam_games = cleaned_steam_games.drop('types', axis=1)

#Get the name to match with a later dataset
cleaned_steam_games.rename(columns={'name':'Name'}, inplace=True)

cleaned_steam_games

,Name,developer,publisher,original_price,Genre1,Genre2,Genre3,Genre4,Genre5,Genre6,...,PopTag11,PopTag12,PopTag13,PopTag14,PopTag15,PopTag16,PopTag17,PopTag18,PopTag19,PopTag20
0,7,Zac,"Zac,Zac",0.00,Adventure,Free to Play,Indie,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,64,rebel rabbit,"indienova,GrabTheGames,indienova,GrabTheGames",2.99,Action,Indie,None,None,None,None,...,Singleplayer,Top-Down Shooter,Arcade,Replay Value,Shooter,Survival,Top-Down,Minimalist,Music,None
2,100$,КиКо,"КиКо,КиКо",1.98,Indie,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,€100,КиКо,"КиКо,КиКо",1.98,Casual,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,103,Dystopia Interactive,"Dystopia Interactive,Dystopia Interactive",9.99,Adventure,Casual,Indie,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30239,Forza Horizon 4,Playground Games,Xbox Game Studios,59.99,Racing,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
30240,STAR WARS™: The Old Republic™,Broadsword,Electronic Arts,0.00,Massively Multiplayer,RPG,Free to Play,None,None,None,...,None,None,None,None,None,None,None,None,None,None
30241,Grounded,Obsidian Entertainment,Xbox Game Studios,39.99,Action,Adventure,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
30242,Stumble Guys,Scopely,Scopely,0.00,Action,Casual,Indie,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [66]:
#Get the most frequent genres
genre_df_copy = pd.DataFrame.copy(genre_df)
genre_df_copy = genre_df_copy.drop('name', axis=1)

melted_genre = pd.melt(genre_df_copy, var_name='Column', value_name='Value')
genre_counts = melted_genre['Value'].value_counts().reset_index()
genre_counts.columns = ['Value','Count']
genre_counts

,Value,Count
0,Indie,18245
1,Action,12659
2,Adventure,9863
3,Casual,9676
4,Simulation,6466
5,Strategy,6019
6,RPG,4923
7,Early Access,2381
8,Free to Play,1958
9,Sports,1459


In [67]:
#Get the most frequent popular tags
pop_tags_copy = pd.DataFrame.copy(popular_tags_df)
pop_tags_copy = pop_tags_copy.drop('name', axis=1)

melted_tags = pd.melt(pop_tags_copy, var_name='Column', value_name='Value')
pop_tag_counts = melted_tags['Value'].value_counts().reset_index()
pop_tag_counts.columns = ['Value','Count']

pop_tag_counts

,Value,Count
0,Indie,19008
1,Action,13338
2,Adventure,10996
3,Casual,10547
4,Simulation,6975
...,...,...
539,4X,1
540,Arcade,1
541,Crime,1
542,4K,1


In [68]:
#Turn the data from the initial webscraping into a dataframe
game_data = pd.DataFrame(data_list, columns=['Name','Current','24h Peak', 'All-Time Peak'] )
game_data

,Name,Current,24h Peak,All-Time Peak
0,Counter-Strike: Global Offensive,"804,976","1,285,148","1,818,773"
1,Dota 2,"265,236","651,740","1,295,114"
2,Baldur's Gate 3,"178,108","284,503","875,343"
3,Cyberpunk 2077,"158,208","167,465","1,054,388"
4,Apex Legends,"146,867","397,605","624,473"
...,...,...,...,...
195,SCP: Secret Laboratory,"2,587","6,964","11,484"
196,Grounded,"2,542","4,933","32,012"
197,Stumble Guys,"2,476","13,268","43,705"
198,MyDockFinder,"2,452","3,539","3,869"


In [69]:
#Merge the webscraped games with the big game list
merged_webscraped_games = pd.merge(cleaned_steam_games, game_data, on='Name', how='inner')
merged_webscraped_games

,Name,developer,publisher,original_price,Genre1,Genre2,Genre3,Genre4,Genre5,Genre6,...,PopTag14,PopTag15,PopTag16,PopTag17,PopTag18,PopTag19,PopTag20,Current,24h Peak,All-Time Peak
0,7 Days to Die,The Fun Pimps,"The Fun Pimps Entertainment LLC,The Fun Pimps ...",24.99,Action,Adventure,Indie,RPG,Simulation,Strategy,...,Horror,Online Co-Op,FPS,Singleplayer,RPG,Indie,Gore,"16,867","30,889","96,827"
1,Albion Online,Sandbox Interactive GmbH,"Sandbox Interactive GmbH,Sandbox Interactive GmbH",0.00,Free to Play,Massively Multiplayer,RPG,None,None,None,...,Loot,Fantasy,Difficult,Strategy,Action,Singleplayer,Indie,"5,914","11,702","26,148"
2,American Truck Simulator,SCS Software,"SCS Software,SCS Software",19.99,Indie,Simulation,None,None,None,None,...,Adventure,Casual,Family Friendly,Management,Indie,Third Person,Action,"4,370","7,121","19,153"
3,Among Us,Innersloth,"Innersloth,Innersloth",4.99,Casual,None,None,None,None,None,...,Minigames,Aliens,Top-Down,Sci-fi,Cartoony,Free to Play,None,"2,778","6,738","447,476"
4,Apex Legends,Respawn Entertainment,Electronic Arts,0.00,Action,Adventure,Free to Play,None,None,None,...,None,None,None,None,None,None,None,"146,867","397,605","624,473"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,Forza Horizon 4,Playground Games,Xbox Game Studios,59.99,Racing,None,None,None,None,None,...,None,None,None,None,None,None,None,"2,659","7,831","40,399"
194,STAR WARS™: The Old Republic™,Broadsword,Electronic Arts,0.00,Massively Multiplayer,RPG,Free to Play,None,None,None,...,None,None,None,None,None,None,None,"2,654","5,525","34,210"
195,Grounded,Obsidian Entertainment,Xbox Game Studios,39.99,Action,Adventure,None,None,None,None,...,None,None,None,None,None,None,None,"2,542","4,933","32,012"
196,Stumble Guys,Scopely,Scopely,0.00,Action,Casual,Indie,None,None,None,...,None,None,None,None,None,None,None,"2,476","13,268","43,705"
